In [ ]:
# Core
import json
from pathlib import Path

# Plotting
import matplotlib.pyplot as plt

# Neuroimaging
import nibabel as nib
import numpy as np

# Data handling
import pandas as pd

In [ ]:
def find_t1w_images(bids_root):
    bids_root = Path(bids_root)
    t1w_files = list(bids_root.rglob("*T1*.nii*"))
    return sorted(t1w_files)


def load_json_sidecar(nifti_path):
    json_path = nifti_path.with_suffix("").with_suffix(".json")
    if json_path.exists():
        with open(json_path, "r") as f:
            return json.load(f)
    return {}


def extract_t1w_stats(nifti_path):
    try:
        img = nib.load(str(nifti_path))
    except Exception as e:
        return {"path": str(nifti_path), "error": str(e)}
    header = img.header
    sidecar = load_json_sidecar(nifti_path)

    # Dimensions
    dims = img.shape
    voxel_sizes = header.get_zooms()[:3]

    # File size (MB)
    file_size_mb = nifti_path.stat().st_size / (1024**2)

    stats = {
        "subject": nifti_path.name.split("_")[0],
        "session": next((p for p in nifti_path.parts if p.startswith("ses-")), None),
        "path": str(nifti_path),
        "dim_x": dims[0],
        "dim_y": dims[1],
        "dim_z": dims[2],
        "voxel_x_mm": voxel_sizes[0],
        "voxel_y_mm": voxel_sizes[1],
        "voxel_z_mm": voxel_sizes[2],
        "file_size_mb": file_size_mb,
        # Metadata (JSON sidecar)
        "tesla": sidecar.get("MagneticFieldStrength"),
        "manufacturer": sidecar.get("Manufacturer"),
        "model": sidecar.get("ManufacturersModelName"),
        "TR": sidecar.get("RepetitionTime"),
        "TE": sidecar.get("EchoTime"),
        "flip_angle": sidecar.get("FlipAngle"),
    }

    return stats


def efc(img, framemask=None, decimals=4):
    r"""
    Calculate the :abbr:`EFC (Entropy Focus Criterion)` [Atkinson1997]_.
    Uses the Shannon entropy of voxel intensities as an indication of ghosting
    and blurring induced by head motion. A range of low values is better,
    with EFC = 0 for all the energy concentrated in one pixel.

    .. math::

        \text{E} = - \sum_{j=1}^N \frac{x_j}{x_\text{max}}
        \ln \left[\frac{x_j}{x_\text{max}}\right]

    with :math:`x_\text{max} = \sqrt{\sum_{j=1}^N x^2_j}`.

    The original equation is normalized by the maximum entropy, so that the
    :abbr:`EFC (Entropy Focus Criterion)` can be compared across images with
    different dimensions:

    .. math::

        \text{EFC} = \left( \frac{N}{\sqrt{N}} \, \log{\sqrt{N}^{-1}} \right) \text{E}

    :param numpy.ndarray img: input data
    :param numpy.ndarray framemask: a mask of empty voxels inserted after a rotation of
      data

    """

    if framemask is None:
        framemask = np.zeros_like(img, dtype=np.uint8)

    n_vox = np.sum(1 - framemask)
    # Calculate the maximum value of the EFC (which occurs any time all
    # voxels have the same value)
    efc_max = 1.0 * n_vox * (1.0 / np.sqrt(n_vox)) * np.log(1.0 / np.sqrt(n_vox))

    # Calculate the total image energy
    b_max = np.sqrt((img[framemask == 0] ** 2).sum())

    # Calculate EFC (add 1e-16 to the image data to keep log happy)
    return round(
        float(
            (1.0 / efc_max)
            * np.sum(
                (img[framemask == 0] / b_max)
                * np.log((img[framemask == 0] + 1e-16) / b_max)
            ),
        ),
        decimals,
    )


def plot_histogram(df, key, bins=20, dropna=True, eps=1e-8):
    if key not in df.columns:
        raise ValueError(f"Key '{key}' not found in dataframe")

    data = df[key]

    if dropna:
        data = data.dropna()

    if len(data) == 0:
        print(f"[WARN] No data available for '{key}'")
        return

    # Attempt numeric conversion
    is_numeric = True
    try:
        data = pd.to_numeric(data)
    except (ValueError, TypeError):
        is_numeric = False

    plt.figure()

    if is_numeric:
        data_min = data.min()
        data_max = data.max()

        # Zero or near-zero range → constant value
        if np.isclose(data_min, data_max, atol=eps):
            plt.bar([str(round(float(data_min), 4))], [len(data)])
            plt.xlabel(key)
            plt.ylabel("Count")
            plt.title(f"{key} (constant value)")
        else:
            # Safe bin count
            effective_bins = min(bins, len(data))
            plt.hist(data, bins=effective_bins)
            plt.xlabel(key)
            plt.ylabel("Count")
            plt.title(f"Histogram of {key}")
    else:
        # Categorical fallback
        counts = data.value_counts()
        plt.bar(counts.index.astype(str), counts.values)
        plt.xlabel(key)
        plt.ylabel("Count")
        plt.title(f"Distribution of {key}")
        plt.xticks(rotation=45, ha="right")

    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
bids_path = "/run/media/falconnier/Elements/datasets/BIDS_datasets/HypnosisBarrios"  # <-- CHANGE THIS
bids_path = "/home/falconnier/Downloads/CamCAN"  # <-- CHANGE THIS

t1w_files = find_t1w_images(bids_path)[:15]
print(f"Found {len(t1w_files)} T1w images")

records = []
for t1w in t1w_files:
    records.append(extract_t1w_stats(t1w))

df = pd.DataFrame(records)
df


In [ ]:
# # Compute QC metrics that do not require any mask or preprocessing
# from scipy.stats import kurtosis, skew


# def compute_maskless_metrics(nifti_path, hist_bins=256):
#     img = nib.load(str(nifti_path)).get_fdata()
#     data = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0).reshape(-1)
#     # keep only finite values
#     data = data[np.isfinite(data)]
#     if data.size == 0:
#         return {"path": str(nifti_path), "error": "empty image"}

#     nz = data[data != 0]
#     # robust mad (median absolute deviation)
#     mad_val = float(np.median(np.abs(data - np.median(data))))

#     # histogram-based entropy
#     hist, _ = np.histogram(data, bins=hist_bins, density=True)
#     probs = hist[hist > 0]
#     entropy = float(-np.sum(probs * np.log2(probs))) if probs.size else 0.0

#     out = {
#         "path": str(nifti_path),
#         "n_voxels": int(data.size),
#         "n_nonzero": int((data != 0).sum()),
#         "nonzero_frac": float((data != 0).mean()),
#         "mean": float(np.mean(data)),
#         "median": float(np.median(data)),
#         "std": float(np.std(data)),
#         "mad": mad_val,
#         "kurtosis": float(kurtosis(data)),
#         "skewness": float(skew(data)),
#         "p01": float(np.percentile(data, 1)),
#         "p05": float(np.percentile(data, 5)),
#         "p95": float(np.percentile(data, 95)),
#         "p99": float(np.percentile(data, 99)),
#         "entropy": entropy,
#     }
#     # EFC from anatomical (works on whole image)
#     try:
#         out["efc"] = float(efc(img))
#     except Exception:
#         out["efc"] = None

#     return out


# # Compute for discovered T1w files and merge with existing dataframe
# qc_records = [compute_maskless_metrics(p) for p in t1w_files]
# df_qc_maskless = pd.DataFrame(qc_records)
# df = df.merge(df_qc_maskless, on="path", how="left")
# df.head()
# df_qc_maskless.describe()
# print(df_qc_maskless.loc[df_qc_maskless["efc"].idxmin(), "path"])
# print(df_qc_maskless.loc[df_qc_maskless["efc"].idxmax(), "path"])
# print(df_qc_maskless.loc[df_qc_maskless["entropy"].idxmin(), "path"])
# print(df_qc_maskless.loc[df_qc_maskless["entropy"].idxmax(), "path"])

In [ ]:
summary = {
    "n_subjects": df["subject"].nunique(),
    "n_sessions": df["session"].nunique(),
    "n_t1w": len(df),
    "voxel_sizes_mm": df[["voxel_x_mm", "voxel_y_mm", "voxel_z_mm"]].drop_duplicates(),
    "tesla_values": df["tesla"].value_counts(dropna=False),
    "manufacturers": df["manufacturer"].value_counts(dropna=False),
}

summary


In [ ]:
df[["voxel_x_mm", "voxel_y_mm", "voxel_z_mm"]].describe()

In [ ]:
# Detect mixed resolutions
df.groupby(["voxel_x_mm", "voxel_y_mm", "voxel_z_mm"]).size()


In [ ]:
# Detect scanner heterogeneity
df.groupby(["manufacturer", "model", "tesla"]).size()


In [ ]:
plot_histogram(df, "voxel_z_mm")
plot_histogram(df, "voxel_x_mm")
plot_histogram(df, "voxel_y_mm")
plot_histogram(df, "file_size_mb")
plot_histogram(df, "tesla", bins=5)
plot_histogram(df, "dim_z")